In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
from dataclasses import dataclass, field
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import networkx as nx
from matplotlib.colors import ListedColormap
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150,
                     'axes.titlesize': 'large', 'axes.labelsize': 'medium',
                     'xtick.labelsize': 'small', 'ytick.labelsize': 'small'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Advanced Agent-Based Modeling (ABM).")

# Chapter 10.1: Agent-Based Models: A "Bottom-Up" Approach to Economic Complexity

---

### Table of Contents
1.  [**The Philosophy and Building Blocks of ABM**](#philosophy)
    - [Emergence, Heterogeneity, and Bounded Rationality](#concepts)
    - [The ODD Protocol for Describing ABMs](#odd)
2.  [**Foundational Model 1: Schelling's Segregation Model**](#schelling)
    - [Emergence and Tipping Points](#schelling-analysis)
3.  [**Foundational Model 2: Axelrod's Tournament and the Evolution of Cooperation**](#axelrod)
    - [Implementing the Iterated Prisoner's Dilemma](#axelrod-impl)
4.  [**Application 1: Financial Market ABMs**](#finance-abm)
    - [The Kirman (1993) "Ants" Model of Herd Behavior](#kirman)
    - [An Artificial Stock Market (Lux & Marchesi)](#asm)
5.  [**Application 2: A Simple Agent-Based Macroeconomic Model**](#macro-abm)
    - [Generating Endogenous Business Cycles](#macro-impl)
6.  [**Strengths, Weaknesses, and Validation**](#validation)
7.  [**Summary and Key Takeaways**](#summary)
8.  [**Exercises**](#exercises)

<a id='philosophy'></a>
## 1. The Philosophy and Building Blocks of ABM

The macroeconomic models explored so far are masterpieces of the **"top-down"** or **deductive** approach. They typically feature a high degree of aggregation—often collapsing the behavior of millions of diverse individuals into a single, hyper-rational **representative agent**—to solve for the unique, stable **equilibrium** of the system. This approach, pioneered by figures like Robert Lucas and Edward Prescott, brought mathematical rigor to macroeconomics but relies on strong assumptions.

This methodology is powerful for analyzing steady-states and the effects of aggregate shocks, but it is less suited to explaining phenomena rooted in **heterogeneity**, **complex network interactions**, and **out-of-equilibrium dynamics**. How do financial crises cascade through the system? How do market bubbles form and burst? How do patterns of extreme wealth inequality arise? These are questions about the *process* of economic dynamics, not just the final destination.

**Agent-Based Modeling (ABM)**, with intellectual roots in the complexity science championed by the Santa Fe Institute, offers a radically different, **"bottom-up"** or **generative** philosophy. Instead of assuming an aggregate equilibrium, an ABM seeks to *generate* aggregate patterns from the ground up, a concept often framed by Joshua Epstein's famous question: **"Can you grow it?"** The goal is to see if complex macroeconomic phenomena can emerge from the repeated interaction of simple, boundedly rational agents.

<a id='concepts'></a>
### Key Concepts: Emergence, Heterogeneity, and Bounded Rationality

The ABM approach is built on a few foundational pillars that distinguish it from traditional methods:
1.  **Heterogeneity:** Agents are not identical clones. They can have different attributes (e.g., wealth, risk tolerance), possess different information, and follow different behavioral rules. This diversity is not just noise; it is a primary driver of complex dynamics that are averaged away in representative agent models.
2.  **Bounded Rationality:** Agents are not hyper-rational optimizers with perfect foresight. They often follow simple heuristics or rules of thumb (e.g., "imitate your successful neighbors," "sell if the price drops 10%"). This concept, largely developed by Nobel laureate Herbert Simon, posits that decision-making is constrained by limited information, cognitive capacity, and time. It is a more psychologically plausible description of human behavior.
3.  **Local Interaction:** Agents interact directly with a limited set of other agents (their "neighbors" on a network) or their local environment. There is no central "Walrasian auctioneer" coordinating global equilibrium. This idea has intellectual origins in the work on **cellular automata** by mathematicians like John von Neumann and Stanisław Ulam in the 1940s, who showed that complex global patterns could arise from simple local rules.
4.  **Emergence:** This is the philosophical core of ABM. Macroeconomic patterns (e.g., business cycles, market crashes, social norms) are not imposed on the model by assumption but are **emergent properties** that arise from the repeated, local, interactions of the heterogeneous agents. The whole becomes qualitatively different and often more complex than the sum of its parts. A classic example is a flock of birds: no single bird is in charge, yet the flock moves as a coherent, intelligent entity. 

![A diagram illustrating emergence, where local interactions between simple agents lead to complex, unpredictable global patterns.](../images/10-Specialized-Models/emergence_diagram.png)

<a id='odd'></a>
### The ODD Protocol for Describing ABMs

Due to their complexity, documenting ABMs in a standardized way is crucial for reproducibility. The **ODD (Overview, Design Concepts, and Details) protocol** is the standard framework for this purpose. It provides a structured format for describing the model's purpose, components, and processes.

**1. Overview**
- **Purpose:** What is the model intended to explain or explore?
- **Entities, State Variables, and Scales:** What are the agents and other entities in the model? What are their attributes? What are the spatial and temporal scales?
- **Process Overview and Scheduling:** What are the basic actions agents perform, and in what order are they executed in each time step?

**2. Design Concepts**
- **Basic Principles:** What are the core theoretical principles underlying the model's design?
- **Emergence:** What key results are expected to emerge from the simulation?
- **Adaptation:** How do agents change their behavior over time?
- **Objectives:** What goals, if any, do the agents have?
- **Learning:** How do agents update their internal models or rules?
- **Sensing:** What can agents observe about their environment and other agents?
- **Interaction:** How do agents interact with each other?
- **Stochasticity:** Where is randomness used in the model?

**3. Details**
- **Initialization:** What is the initial state of the world? How are agent attributes and network connections set up?
- **Input Data:** Does the model use any external data?
- **Submodels:** What are the precise mathematical or logical formulations for the processes within the model (e.g., the exact rule for an agent's decision to move)?

Throughout this notebook, we will implicitly follow the ODD structure when describing each model.

<a id='schelling'></a>
## 2. Foundational Model 1: Schelling's Segregation Model

Thomas Schelling's 1971 model, developed in his paper "Dynamic Models of Segregation," is a canonical ABM that provides a stunning and often troubling demonstration of **emergence**. It shows how stark, macro-level patterns of residential segregation can arise *even when no single individual is actively intolerant*. The model reveals a tragic disconnect between individual intentions and collective outcomes.

**The Core Idea:** Schelling wanted to understand if severe segregation could be the result of something other than explicit racism or centrally-planned housing policies. His hypothesis was that it could emerge from the cumulative effect of many individuals making seemingly innocuous choices based on a mild preference for not being in a minority.

--- 

**ODD Specification:**
- **Purpose:** To demonstrate how collective segregation patterns can emerge from weak, local, individual preferences regarding the composition of one's neighborhood.
- **Entities:** Agents of different types (e.g., 'crimson' and 'royalblue') and a grid representing a city, with some empty patches.
- **State Variables:** Each agent has a `type` and a `location` (a cell on the grid). Each grid cell has a `status` (empty or occupied by an agent of a certain type).
- **Process Overview:** 
  1. In each time step, identify all "unhappy" agents.
  2. An agent is **unhappy** if the proportion of their eight immediate neighbors who are of the same type is below their `tolerance` threshold.
  3. A random subset of these unhappy agents is then moved to random empty locations on the grid.
  4. This process repeats until no agents are unhappy, or a maximum number of iterations is reached.
- **Design Concepts:** The core concept is **emergence**. There is no central planner, no discriminatory laws, no explicit rule for segregation. The macro-pattern of segregated clusters emerges purely from the repeated, local, boundedly rational decisions of the agents. Agents are **boundedly rational** because they follow a simple happiness rule rather than solving a global optimization problem about where best to live.
- **Details:** The model is initialized with a random distribution of agents on the grid. The key parameter that we will explore is the `tolerance` threshold.

In [ ]:
sec("Interactive Schelling Segregation Model")
class SchellingModel:
    def __init__(self, size=50, empty_ratio=0.1, tolerance=0.4, n_types=3):
        self.size, self.tolerance, self.n_types = size, tolerance, n_types; self.segregation_history = []
        self.cmap = ListedColormap(['#f0f0f0', 'crimson', 'royalblue', 'forestgreen', 'darkorange'][:n_types + 1])
        n_cells = size**2; n_agents = int(n_cells * (1 - empty_ratio)); pop_list = np.array_split(np.arange(n_agents), n_types)
        pop_list = sum([[i+1]*len(arr) for i, arr in enumerate(pop_list)], []); population = np.array([0]*(n_cells - len(pop_list)) + pop_list)
        np.random.shuffle(population); self.grid = population.reshape((size, size)); self._calculate_segregation()
    def _get_neighbors(self, r, c): return [self.grid[(r+dr)%self.size, (c+dc)%self.size] for dr in [-1,0,1] for dc in [-1,0,1] if not (dr==0 and dc==0)]
    def _calculate_segregation(self):
        similarities = [np.mean([n == self.grid[r,c] for n in self._get_neighbors(r,c) if n!=0]) for r, c in np.argwhere(self.grid > 0) if any(n!=0 for n in self._get_neighbors(r,c))]
        self.segregation_history.append(np.mean(similarities) if similarities else 1.0)
    def step(self):
        unhappy_agents = [loc for loc in np.argwhere(self.grid > 0) if np.mean([n == self.grid[tuple(loc)] for n in self._get_neighbors(*loc) if n != 0]) < self.tolerance]
        if not unhappy_agents: return 0
        empty_cells = list(zip(*np.where(self.grid == 0))); np.random.shuffle(unhappy_agents); np.random.shuffle(empty_cells)
        num_to_move = min(len(unhappy_agents), len(empty_cells))
        for i in range(num_to_move):
            self.grid[tuple(empty_cells[i])] = self.grid[tuple(unhappy_agents[i])]; self.grid[tuple(unhappy_agents[i])] = 0
        self._calculate_segregation(); return num_to_move

def run_and_plot_schelling(tolerance=0.4):
    model = SchellingModel(tolerance=tolerance, n_types=3, size=50); initial_grid = model.grid.copy()
    for i in range(150): 
        if model.step() == 0: break
    note(f"Tolerance {tolerance:.2f}: Converged after {i+1} iterations."); fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    axes[0].imshow(initial_grid, cmap=model.cmap, interpolation='nearest'); axes[0].set_title('Initial State')
    axes[1].imshow(model.grid, cmap=model.cmap, interpolation='nearest'); axes[1].set_title('Final State')
    for ax in [axes[0], axes[1]]: ax.set_xticks([]); ax.set_yticks([])
    axes[2].plot(model.segregation_history, '-o', lw=2); axes[2].set_title('Segregation Index'); axes[2].set(xlabel='Iteration', ylabel='Avg. Similarity', ylim=(0, 1));
    plt.tight_layout(); plt.show()

widgets.interact(run_and_plot_schelling, tolerance=widgets.FloatSlider(min=0.1, max=0.8, step=0.05, value=0.4, description='Tolerance'));

<a id='schelling-analysis'></a>
### Emergence and Tipping Points
The interactive simulation reveals a crucial insight: there is a **tipping point**. Below a certain tolerance level (e.g., < 0.33), the society remains integrated. But once the tolerance crosses this critical threshold, a small change in individual preference leads to a dramatic, nonlinear shift in the collective outcome, resulting in a highly segregated equilibrium. This emergent result is a hallmark of complex systems and demonstrates how market-like mechanisms can fail to achieve socially optimal outcomes, as the final segregated state is often Pareto-dominated.

<a id='axelrod'></a>
## 3. Foundational Model 2: Axelrod's Tournament and the Evolution of Cooperation

Why does cooperation exist in a world of self-interested actors? This question has puzzled thinkers for centuries. In the late 1970s and early 1980s, political scientist Robert Axelrod provided a powerful answer using an agent-based approach. He hosted a computer tournament where experts from various fields submitted strategies (agents) for the **Iterated Prisoner's Dilemma (IPD)** game.

The Prisoner's Dilemma is a cornerstone of game theory. In a single round, the dominant strategy for a rational actor is to **Defect**, leading to a collectively suboptimal outcome. However, Axelrod's tournament explored the *iterated* version, where the same two players face each other multiple times. This introduces the concepts of **reputation, retaliation, and forgiveness**, fundamentally changing the strategic landscape.

Axelrod's key finding was that the simplest strategy submitted, **Tit-for-Tat** (start by cooperating, then copy your opponent's previous move), consistently won the tournament. It succeeded not by being the most aggressive, but by embodying a few key principles:
- **Nice:** It was never the first to defect, opening the door for mutual cooperation.
- **Retaliatory:** It immediately punished any defection, preventing exploitation.
- **Forgiving:** It was willing to return to cooperation if the opponent did, restoring trust.
- **Clear:** Its simple rule was easy for other strategies to recognize and adapt to, fostering predictable and stable interactions.

<a id='axelrod-impl'></a>
### Implementing the Iterated Prisoner's Dilemma
We can replicate Axelrod's findings by creating a population of agents with different strategies and simulating their evolution. In an 'ecological' simulation, successful strategies (those that accumulate the highest payoffs) are more likely to "reproduce," meaning their proportion in the population grows over time, while unsuccessful strategies die out.

In [ ]:
sec("Axelrod's Tournament Simulation")

# --- 1. Define Agent Strategies ---
# Each strategy is a function that takes its own history and the opponent's history
# and returns either 'C' (Cooperate) or 'D' (Defect).

def tit_for_tat(my_history, their_history):
    """Starts with cooperation, then mirrors the opponent's last move."""
    return 'C' if not their_history else their_history[-1]

def always_defect(my_history, their_history):
    """Always defects, regardless of history."""
    return 'D'

def always_cooperate(my_history, their_history):
    """Always cooperates, regardless of history. A naive strategy."""
    return 'C'

def grim_trigger(my_history, their_history):
    """Cooperates until the opponent defects once, then defects forever."""
    return 'D' if 'D' in their_history else 'C'

def random_move(my_history, their_history):
    """Makes a random move. Unpredictable."""
    return random.choice(['C', 'D'])

@dataclass
class AxelrodAgent:
    strategy_func: callable
    name: str
    history: list = field(default_factory=list)
    score: float = 0.0

# --- 2. The Tournament and Evolution Engine ---
class AxelrodTournament:
    def __init__(self, population_counts):
        """Initializes the tournament with a given population mix."""
        self.strategies = {'Tit-for-Tat': tit_for_tat, 'Always Defect': always_defect, 
                           'Always Cooperate': always_cooperate, 'Grim Trigger': grim_trigger, 'Random': random_move}
        self.population = []
        for name, count in population_counts.items():
            self.population.extend([AxelrodAgent(self.strategies[name], name) for _ in range(count)])
        # Payoff matrix: (my_score, their_score)
        self.payoffs = {('C', 'C'): (3, 3), ('C', 'D'): (0, 5), ('D', 'C'): (5, 0), ('D', 'D'): (1, 1)}

    def run_round_robin(self, n_rounds=10):
        """Each agent plays against every other agent in the population."""
        for i, p1 in enumerate(self.population):
            for j, p2 in enumerate(self.population[i+1:]):
                # Reset histories for this new matchup
                p1.history, p2.history = [], []
                for _ in range(n_rounds):
                    move1 = p1.strategy_func(p1.history, p2.history)
                    move2 = p2.strategy_func(p2.history, p1.history)
                    p1.history.append(move1); p2.history.append(move2)
                    score1, score2 = self.payoffs[(move1, move2)]
                    p1.score += score1; p2.score += score2

    def evolve(self):
        """Replaces the old generation with a new one based on fitness (score)."""
        total_score = sum(a.score for a in self.population)
        if total_score == 0: return # Avoid division by zero in the initial stages
        
        # Agents with higher scores have a higher probability of being selected for the next generation
        fitness = [a.score / total_score for a in self.population]
        new_population_agents = np.random.choice(self.population, size=len(self.population), p=fitness, replace=True)
        
        # Create a new population of fresh agents based on the selected strategies
        self.population = [AxelrodAgent(a.strategy_func, a.name) for a in new_population_agents]
        
    def run_simulation(self, n_generations=50):
        history = []
        for _ in range(n_generations):
            counts = {name: 0 for name in self.strategies}
            for agent in self.population: counts[agent.name] += 1
            history.append(counts)
            self.run_round_robin()
            self.evolve()
        return pd.DataFrame(history)

# --- Run and Plot ---
initial_counts = {'Tit-for-Tat': 25, 'Always Defect': 25, 'Always Cooperate': 25, 'Grim Trigger': 25}
tournament = AxelrodTournament(initial_counts)
history_df = tournament.run_simulation(n_generations=30)

fig, ax = plt.subplots(figsize=(14, 8))
history_df.plot(ax=ax, lw=3)
ax.set_title("Figure 2: Evolution of Strategies in Axelrod's Tournament", fontsize=18)
ax.set_xlabel('Generation'); ax.set_ylabel('Number of Agents')
ax.legend(title='Strategy'); ax.grid(True)
plt.show()

note("The simulation shows that 'nice' but retaliatory strategies like Tit-for-Tat and Grim Trigger tend to thrive in an evolutionary setting. \n\n- **Initial Chaos:** Initially, 'Always Defect' performs well by exploiting the naive 'Always Cooperate' agents, driving them to extinction.\n- **Rise of Cooperation:** However, once the purely naive agents are gone, 'Always Defect' can only prey on itself, leading to low scores. This allows Tit-for-Tat and Grim Trigger to outperform it by cooperating with each other.\n- **Stable Equilibrium:** The system converges to a state dominated by cooperative-yet-punishing strategies, providing a powerful model for how cooperation can emerge and sustain itself in a world of self-interested actors.")

<a id='finance-abm'></a>
## 4. Application 1: Financial Market ABMs

ABMs have become a powerful tool for studying financial markets because they can generate many of the well-documented **"stylized facts"** of financial time series that are difficult to explain with standard representative-agent, efficient-market models. These facts include:
- **Fat Tails:** The distribution of asset returns has more extreme values (crashes and booms) than a normal distribution would suggest.
- **Volatility Clustering:** Periods of high volatility are followed by more high volatility, and periods of calm are followed by calm. Volatility is not constant.
- **Bubbles and Crashes:** Asset prices can deviate significantly from their fundamental values for extended periods, driven by social dynamics rather than new information.

<a id='kirman'></a>
### The Kirman (1993) "Ants" Model of Herd Behavior

One of the simplest and most elegant models of emergent herd behavior is Alan Kirman's 1993 model, originally inspired by the foraging behavior of ants. It shows how collective opinion can become highly concentrated and switch regimes, even when there is no underlying fundamental information driving the process. The model has become a workhorse in behavioral finance.

**The Setup:** Imagine a market with a fixed number of traders. At any time, a trader can be in one of two states: **optimistic** (believing the market will go up) or **pessimistic** (believing it will go down). They change their state based on two simple, psychologically plausible factors:
1.  **Idiosyncratic Shocks:** An agent might randomly switch their opinion due to some private signal or personal re-evaluation (with a small probability, $\epsilon$). This represents individual thinking.
2.  **Social Influence (Herding):** An agent meets another agent and may be convinced to adopt their opinion (with probability $h$). This represents social pressure or contagion.

The key insight is that even with no underlying "fundamental" value, the model can generate periods where one opinion becomes almost completely dominant purely through social contagion. This mimics the herd behavior and sudden sentiment shifts seen in real financial markets.

In [ ]:
sec("Kirman's Ant Model of Herd Behavior")

class KirmanAntsModel:
    def __init__(self, N=100, epsilon=0.05, h=0.2):
        """
        Initializes the model.
        Args:
            N (int): Total number of agents in the market.
            epsilon (float): Probability of an idiosyncratic opinion switch.
            h (float): Probability of being influenced by another agent (herding).
        """
        self.N, self.epsilon, self.h = N, epsilon, h
        self.n_optimists = N // 2 # Start with a 50/50 split
        
    def step(self):
        """Executes a single time step of the simulation."""
        # 1. Pick a random agent to make a decision
        agent_idx = random.randint(0, self.N - 1)
        is_optimist = agent_idx < self.n_optimists # A simple way to check the agent's type
        
        # 2. Check for an idiosyncratic switch (individual thinking)
        if random.random() < self.epsilon:
            # The agent flips their opinion regardless of others
            if is_optimist: self.n_optimists -= 1
            else: self.n_optimists += 1
            return
        
        # 3. Check for a social interaction (herding)
        if random.random() < self.h:
            # Pick another agent at random to interact with
            other_idx = random.randint(0, self.N - 1)
            other_is_optimist = other_idx < self.n_optimists
            
            # If the two agents have different opinions, the first agent adopts the second's opinion
            if is_optimist and not other_is_optimist: self.n_optimists -= 1
            elif not is_optimist and other_is_optimist: self.n_optimists += 1
            
    def run(self, n_steps=5000):
        history = np.zeros(n_steps)
        for t in range(n_steps):
            self.step()
            history[t] = self.n_optimists / self.N
        return history

# --- Run and Plot ---
kirman_model = KirmanAntsModel(N=100, epsilon=0.01, h=0.3)
optimist_fraction = kirman_model.run(n_steps=10000)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), gridspec_kw={'height_ratios': [3, 1]})
fig.suptitle("Figure 3: Kirman's Model of Herding", fontsize=18)
ax1.plot(optimist_fraction, label='Fraction of Optimists')
ax1.set_title('Time Series of Agent States')
ax1.set_ylabel('Fraction')
ax1.set_ylim(0, 1)

ax2.hist(optimist_fraction, bins=50, density=True)
ax2.set_title('Distribution of States')
ax2.set_xlabel('Fraction of Optimists')
ax2.set_ylabel('Density')

for ax in [ax1, ax2]: ax.grid(True)
plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

note("The simulation results reveal two key features:\n\n1.  **Time Series:** The fraction of optimists is not random noise. It persists for long periods near the boundaries (0 or 1), with occasional, rapid switches between these states of consensus.\n2.  **Bimodal Distribution:** The histogram shows that the system spends most of its time in states where one opinion is overwhelmingly dominant (most agents are optimists or most are pessimists) and very little time in a state of disagreement.\n\nThis emergent behavior is driven by the **positive feedback** of the herding term ($h$). When a majority of agents become optimistic by chance, it becomes increasingly likely that any given pessimist will meet an optimist and be converted, reinforcing the majority view. The idiosyncratic term ($\epsilon$) acts as a small, persistent disruptive force that is necessary to eventually knock the system out of one consensus and into the other.")

<a id='asm'></a>
### An Artificial Stock Market (Lux & Marchesi)

Building on the idea of interacting agent types, we can construct a simple Artificial Stock Market (ASM). This model, in the spirit of Lux & Marchesi (1999), generates endogenous bubbles, crashes, and volatility clustering from the interaction of two types of traders:
1.  **Fundamentalists:** They believe the price will revert to a fundamental value $P_f$. Their demand for the stock is proportional to the perceived mispricing, $(P_f - P_t)$.
2.  **Chartists (or Noise Traders):** They extrapolate past trends. Their demand is proportional to the last observed price change, $(P_t - P_{t-1})$.

Agents can switch between these strategies based on which is performing better. The price is set by a **market maker** who adjusts the price in response to excess demand.
$$ P_{t+1} = P_t + \beta (D_t - S_t) $$
This feedback loop—where prices affect strategies, which affect demand, which in turn affects prices—is the engine of the model's complex dynamics.

In [ ]:
sec("Interactive Artificial Stock Market Simulation")

class ArtificialStockMarket:
    def __init__(self, n_agents=100, p_fundamental=100.0, beta=0.5, v=1.0):
        self.n_agents, self.p_fundamental, self.beta, self.v = n_agents, p_fundamental, beta, v
        self.prices = [p_fundamental, p_fundamental] # Start with two prices for trend calculation
        self.n_chartists = n_agents // 2
        self.chartist_fraction_history = [0.5, 0.5]

    def step(self):
        n_fundamentalists = self.n_agents - self.n_chartists
        last_price, prev_price = self.prices[-1], self.prices[-2]
        demand_f = n_fundamentalists * (self.p_fundamental - last_price)
        demand_c = self.n_chartists * (last_price - prev_price)
        new_price = last_price + self.beta * (demand_f + demand_c) / self.n_agents
        self.prices.append(new_price)
        profit_f = (new_price - last_price) * (self.p_fundamental - last_price)
        profit_c = (new_price - last_price) * (last_price - prev_price)
        attraction = profit_c - profit_f
        frac_chartists = 1 / (1 + np.exp(-self.v * attraction))
        self.n_chartists = int(self.n_agents * frac_chartists)
        self.chartist_fraction_history.append(self.n_chartists / self.n_agents)

    def run(self, n_steps=500):
        for _ in range(n_steps): self.step()
        return self.prices, self.chartist_fraction_history

def interactive_asm_plot(beta=0.3, v=0.1):
    asm = ArtificialStockMarket(beta=beta, v=v)
    prices, chartist_fractions = asm.run(n_steps=500)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    fig.suptitle('Figure 4: An Artificial Stock Market', fontsize=18)
    ax1.plot(prices, label='Market Price', lw=2); ax1.axhline(asm.p_fundamental, color='k', linestyle='--', label='Fundamental Value')
    ax1.set_title('Endogenous Bubbles and Crashes'); ax1.set_ylabel('Price'); ax1.legend(); ax1.grid(True)
    ax2.plot(chartist_fractions, 'r-', label='Fraction of Chartists'); ax2.set_title('Evolution of Trader Population')
    ax2.set_xlabel('Time'); ax2.set_ylabel('Fraction'); ax2.set_ylim(0, 1); ax2.legend(); ax2.grid(True)
    plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

note("Use the sliders to see how micro-level parameters affect macro-level outcomes. A higher `beta` (price sensitivity) or `v` (strategy switching speed) can lead to more volatile markets.")
widgets.interact(interactive_asm_plot, 
                 beta=widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.3, description='Price Speed (β)'),
                 v=widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description='Switch Speed (v)'));


<a id='macro-abm'></a>
## 5. Application 2: A Simple Agent-Based Macroeconomic Model

A growing body of research uses large-scale ABMs to analyze macroeconomic policy. These models, such as the one developed by Delli Gatti et al. (2011), feature thousands of interacting households, firms, and banks. They can generate endogenous business cycles, financial crises, and other complex dynamics from the interactions of these heterogeneous agents. Unlike DSGE models, they do not assume a single rational expectations equilibrium.

<a id='macro-impl'></a>
### Generating Endogenous Business Cycles
We can build a stylized model to demonstrate this. The model has two main agent types:
- **Households:** They consume a fraction of their income and save the rest. They supply labor to firms.
- **Firms:** They hire workers to produce a consumption good. They set prices based on a markup over their wage costs and adjust production based on demand. Firms that are unprofitable may go bankrupt, while successful firms can expand.

The interaction of these simple, boundedly rational rules can generate complex aggregate dynamics, including self-sustaining business cycles.

In [ ]:
sec("A Simple Agent-Based Macroeconomic Model")

@dataclass
class Household:
    id: int; wealth: float = 10.0; employed: bool = False

@dataclass
class Firm:
    id: int; capital: float = 100.0; employees: list = field(default_factory=list)
    price: float = 1.0; production_target: float = 10.0

class MacroABM:
    def __init__(self, n_households=500, n_firms=50, consumption_propensity=0.8, markup=0.2):
        self.households = [Household(id=i) for i in range(n_households)]
        self.firms = [Firm(id=i) for i in range(n_firms)]
        self.unemployment_rate = 1.0
        self.gdp = 0
        self.params = {'c': consumption_propensity, 'm': markup}
        self.history = {'gdp': [], 'unemployment': []}

    def step(self):
        # 1. Firms set prices and production targets
        avg_wage = 1.0 # Simplified
        for firm in self.firms:
            firm.price = (1 + self.params['m']) * avg_wage
            # Simple adaptive expectations for demand
            firm.production_target = 0.9 * firm.production_target + 0.1 * self.gdp / len(self.firms)
        
        # 2. Labor market
        for h in self.households: h.employed = False
        for f in self.firms: f.employees = []
        
        unemployed = list(self.households)
        random.shuffle(unemployed)
        for firm in self.firms:
            n_hires = int(firm.production_target / 10) # Simple production function
            for _ in range(n_hires):
                if not unemployed: break
                worker = unemployed.pop(); worker.employed = True; firm.employees.append(worker)
        
        self.unemployment_rate = len(unemployed) / len(self.households)
        
        # 3. Production and Income
        self.gdp = 0
        for firm in self.firms:
            production = len(firm.employees) * 10
            self.gdp += production * firm.price
            for worker in firm.employees: worker.wealth += avg_wage * firm.price
            
        # 4. Consumption
        for household in self.households:
            if household.wealth > 0:
                consumption_amount = self.params['c'] * household.wealth
                household.wealth -= consumption_amount
        
        self.history['gdp'].append(self.gdp)
        self.history['unemployment'].append(self.unemployment_rate)

    def run(self, n_steps=200):
        for _ in range(n_steps): self.step()
        return self.history

# --- Run and Plot ---
macro_model = MacroABM()
history = macro_model.run()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
fig.suptitle('Figure 5: An Agent-Based Macroeconomic Model', fontsize=18)

ax1.plot(history['gdp'], label='GDP')
ax1.set_title('Endogenous Business Cycles')
ax1.set_ylabel('Nominal GDP')
ax1.grid(True)

ax2.plot(history['unemployment'], 'r-', label='Unemployment Rate')
ax2.set_title('Labor Market Dynamics')
ax2.set_xlabel('Time')
ax2.set_ylabel('Rate')
ax2.set_ylim(0, 1)
ax2.grid(True)
plt.show()

note("Even this highly stylized model generates persistent fluctuations in GDP and unemployment. The interaction between firms' production decisions and households' income and consumption creates feedback loops that drive the system, producing business-cycle-like dynamics without any external aggregate shocks.")

<a id='network-abm'></a>
## 6. Application 3: Opinion Dynamics on a Network

A key advantage of ABM is the ability to explicitly model interaction structures using networks. Let's demonstrate this with a simple **Voter Model**. Agents (nodes) on a network hold one of two opinions (e.g., 0 or 1). In each step, a random agent is chosen and they adopt the opinion of a randomly chosen neighbor. This simple local rule can lead to global consensus, but the time it takes depends critically on the network structure.

We will set up a **small-world network** (using the Watts-Strogatz model), which is known to have properties often seen in real social networks: high clustering and short average path lengths.

In [ ]:
sec("Voter Model on a Small-World Network")

def setup_voter_model(n_agents=50, n_neighbors=4, p_rewire=0.1):
    G = nx.watts_strogatz_graph(n_agents, n_neighbors, p_rewire, seed=42)
    for i in G.nodes():
        G.nodes[i]['opinion'] = 1 if i < n_agents / 2 else 0
    return G

def voter_step(G):
    voter = random.choice(list(G.nodes()))
    neighbors = list(G.neighbors(voter))
    if neighbors:
        chosen_neighbor = random.choice(neighbors)
        G.nodes[voter]['opinion'] = G.nodes[chosen_neighbor]['opinion']

def update_plot(frame, G, pos, ax):
    ax.clear()
    voter_step(G)
    opinions = [G.nodes[i]['opinion'] for i in G.nodes()]
    colors = ['crimson' if op == 1 else 'royalblue' for op in opinions]
    nx.draw(G, pos=pos, node_color=colors, with_labels=False, ax=ax, node_size=200)
    ax.set_title(f'Voter Model - Iteration: {frame + 1}')

# --- Setup and Run Animation ---
G = setup_voter_model()
pos = nx.spring_layout(G, seed=42)

fig, ax = plt.subplots(figsize=(10, 10))
ani = FuncAnimation(fig, update_plot, frames=150, fargs=(G, pos, ax), interval=100)
plt.close() # Prevents static plot from showing

note("Displaying animation of opinion dynamics. It may take a moment to render. Clusters of opinion form and compete until one dominates.")
display(HTML(ani.to_jshtml()))

<a id='validation'></a>
## 7. Strengths, Weaknesses, and Validation

**Strengths:**
- **Generality:** Can model complex systems with heterogeneity and non-linear interactions that are intractable with analytical methods.
- **Emergence:** Provides a framework for understanding how macro-phenomena arise from micro-level behavior.
- **Flexibility:** Easy to incorporate diverse agent behaviors, network structures, and institutional rules.

**Weaknesses:**
- **"Atheoretical":** A common criticism is that ABMs can become complex collections of ad-hoc rules without strong theoretical discipline.
- **Calibration and Validation:** With many parameters, it's difficult to rigorously calibrate the model to data. Validating that the model's success isn't just due to overfitting is a major challenge (the "curse of dimensionality").
- **Computational Cost:** Large-scale ABMs can be computationally intensive to simulate.

Validation often proceeds via **stylized fact replication**: does the model, for a plausible range of parameters, generate qualitative and quantitative patterns that match those seen in the real world (e.g., fat-tailed returns, volatility clustering)?

<a id='summary'></a>
## 7. Summary and Key Takeaways

This notebook provided a survey of the agent-based modeling approach, a powerful alternative to traditional equilibrium models.
- ABMs are a **bottom-up, generative** method for studying complex systems, built on the principles of **heterogeneity, bounded rationality, and local interaction**.
- Foundational models like **Schelling's segregation model** and **Axelrod's tournament** demonstrate how complex macro-patterns (segregation, cooperation) can **emerge** from simple micro-rules.
- **Financial ABMs**, like the Kirman and Lux-Marchesi models, can replicate key stylized facts of financial markets, such as bubbles, crashes, and fat-tailed returns, by modeling the interaction of agents with different strategies.
- **Macroeconomic ABMs** can generate endogenous business cycles and other aggregate dynamics from the interactions of households and firms, providing insights into out-of-equilibrium phenomena.
- While powerful, ABMs face significant challenges in **calibration and validation**, which remains an active area of research.

<a id='exercises'></a>
## 8. Exercises and Discussion Points

1.  **Mapping the Schelling Tipping Point:** Write a new function that loops through `tolerance` values from 0.2 to 0.6 in steps of 0.05. For each value, run a full Schelling simulation until it converges and store the final segregation index. Plot the final segregation index (y-axis) against the tolerance (x-axis). Describe the resulting curve. Is the relationship linear? What does this plot tell you about the sensitivity of collective outcomes to small changes in individual preferences?

2.  **Robustness of Tit-for-Tat:** Introduce a new strategy to the `AxelrodTournament` called `Tit-for-Two-Tats` (it only defects if the opponent has defected in the *last two* consecutive rounds). First, simulate a population where `Tit-for-Tat` is dominant but a small minority (e.g., 10%) are `Always Defect`. Does `Tit-for-Tat` survive? Now, replace `Tit-for-Tat` with `Tit-for-Two-Tats` and run the same simulation. Is the more forgiving strategy more or less resilient to invasion by defectors? Explain the result.

3.  **Herding vs. Individuality in the Kirman Model:** In the `KirmanAntsModel`, set the herding parameter `h` to a high value (e.g., 0.8) and the idiosyncratic parameter `epsilon` to a very low value (e.g., 0.005). Then do the reverse: set `h` low (0.1) and `epsilon` high (0.2). Generate the time-series and histogram plots for both scenarios. How does the shape of the distribution and the persistence of consensus change in each case? Relate your findings to the tension between social influence and private information in real-world markets.

4.  **Market Maker Speed and Volatility:** In the `ArtificialStockMarket`, the `beta` parameter represents how quickly the market maker adjusts prices to excess demand. Run two simulations: one with a low `beta` (e.g., 0.1) and one with a high `beta` (e.g., 0.8), keeping other parameters constant. How does `beta` affect the size and frequency of bubbles and crashes? Provide an economic intuition for why a faster-reacting market might lead to more, not less, volatility.

5.  **Counter-Cyclical Fiscal Policy in the Macro ABM:** Add a simple government agent to the `MacroABM`. This agent will have a `budget_balance` attribute. In each step, if the economy is in a recession (defined as the unemployment rate being above a threshold, e.g., 7%), the government implements a stimulus: it hires a fixed number of unemployed workers directly (e.g., 5% of the unemployed pool), paying them the average wage. This spending adds to the government's deficit. If not in a recession, the government levies a small tax on all employed workers' income to pay down its debt. Does this simple counter-cyclical policy stabilize GDP and unemployment? Plot the evolution of the government's `budget_balance` over time.